In [6]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'golpes_3_340\images'
output_dir = 'dataug'
label_dir = 'golpes_3_340\labels'

# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    # Random rotation
    angle = np.random.uniform(-10, 10)
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    image = cv2.warpAffine(image, M, (cols, rows))

    # Random horizontal flip
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        bbox[0], bbox[2] = 1 - bbox[2], 1 - bbox[0]  # Adjust bounding box x-coordinates

        # Flip keypoints
        for i in range(0, len(keypoints), 2):
            keypoints[i] = 1 - keypoints[i]

    return image, bbox, keypoints

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_image, augmented_bbox, augmented_keypoints = augment_data(image, bbox, keypoints)

        # Save augmented image
        augmented_image_path = os.path.join(output_dir, 'aug_' + filename)
        cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

        # Save augmented labels
        augmented_label_path = os.path.join(output_dir, 'aug_' + filename[:-4] + '.txt')
        with open(augmented_label_path, 'w') as augmented_label_file:
            augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
            augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")

Data augmentation complete.


In [1]:
import cv2
import numpy as np
import os

def draw_bbox_and_keypoints(image, bbox, keypoints, color=(255, 0, 0)):
    # Draw bounding box
    x, y, w, h = bbox
    pt1 = (int(x * image.shape[1]), int(y * image.shape[0]))
    pt2 = (int((x + w) * image.shape[1]), int((y + h) * image.shape[0]))
    cv2.rectangle(image, pt1, pt2, color, 2)

    # Draw keypoints
    for i in range(0, len(keypoints), 2):
        kp_x = int(keypoints[i] * image.shape[1])
        kp_y = int(keypoints[i + 1] * image.shape[0])
        cv2.circle(image, (kp_x, kp_y), 3, color, -1)

    return image

# Define your input directory
input_dir = 'dataug'

label_dir = 'dataug'

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Draw bounding box and keypoints on the image
        image_with_annotations = draw_bbox_and_keypoints(image.copy(), bbox, keypoints)

        # Display the result
        cv2.imshow('Annotations Verification', cv2.cvtColor(image_with_annotations, cv2.COLOR_RGB2BGR))
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'golpes_3_340\images'
output_dir = 'dataug'
label_dir = 'golpes_3_340\labels'


# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Helper functions for bounding box and keypoints transformations
def flip_bbox(bbox, image_width):
    return [image_width - bbox[2], bbox[1], image_width - bbox[0], bbox[3]]

def rotate_bbox(bbox, angle, image_width, image_height):
    center_x = (bbox[0] + bbox[2]) / 2
    center_y = (bbox[1] + bbox[3]) / 2

    new_center_x = center_x * np.cos(np.radians(angle)) - center_y * np.sin(np.radians(angle))
    new_center_y = center_x * np.sin(np.radians(angle)) + center_y * np.cos(np.radians(angle))

    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]

    new_width = np.abs(width * np.cos(np.radians(angle))) + np.abs(height * np.sin(np.radians(angle)))
    new_height = np.abs(height * np.cos(np.radians(angle))) + np.abs(width * np.sin(np.radians(angle)))

    x_min = max(0, new_center_x - new_width / 2)
    y_min = max(0, new_center_y - new_height / 2)
    x_max = min(image_width, new_center_x + new_width / 2)
    y_max = min(image_height, new_center_y + new_height / 2)

    return [x_min, y_min, x_max, y_max]

def rotate_keypoints(keypoints, angle, image_width, image_height):
    rotated_keypoints = []

    for i in range(0, len(keypoints), 2):
        x = keypoints[i]
        y = keypoints[i + 1]

        new_x = x * np.cos(np.radians(angle)) - y * np.sin(np.radians(angle))
        new_y = x * np.sin(np.radians(angle)) + y * np.cos(np.radians(angle))

        rotated_keypoints.extend([new_x, new_y])

    return rotated_keypoints

def flip_keypoints(keypoints, image_width):
    flipped_keypoints = [image_width - kp if i % 2 == 0 else kp for i, kp in enumerate(keypoints)]
    return flipped_keypoints

def zoom_bbox(bbox, zoom_factor, image_width):
    center_x = (bbox[0] + bbox[2]) / 2

    new_width = (bbox[2] - bbox[0]) * zoom_factor

    x_min = max(0, center_x - new_width / 2)
    x_max = min(image_width, center_x + new_width / 2)

    return [x_min, bbox[1], x_max, bbox[3]]

def zoom_keypoints(keypoints, zoom_factor, image_width):
    center_x = (keypoints[0] + keypoints[2]) / 2

    new_x_0 = center_x - (center_x - keypoints[0]) * zoom_factor
    new_x_2 = center_x + (keypoints[2] - center_x) * zoom_factor

    return [new_x_0, keypoints[1], new_x_2, keypoints[3]]

# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    augmented_data = []

    # Original data
    augmented_data.append((image, bbox, keypoints))

    # Rotate 30°
    rotated_image = cv2.warpAffine(image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), 30, 1), (image.shape[1], image.shape[0]))
    rotated_bbox = rotate_bbox(bbox, 30, image.shape[1], image.shape[0])
    rotated_keypoints = rotate_keypoints(keypoints, 30, image.shape[1], image.shape[0])
    augmented_data.append((rotated_image, rotated_bbox, rotated_keypoints))

    # Rotate -30°
    rotated_image = cv2.warpAffine(image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), -30, 1), (image.shape[1], image.shape[0]))
    rotated_bbox = rotate_bbox(bbox, -30, image.shape[1], image.shape[0])
    rotated_keypoints = rotate_keypoints(keypoints, -30, image.shape[1], image.shape[0])
    augmented_data.append((rotated_image, rotated_bbox, rotated_keypoints))

    # Contrast +50
    contrast_image = cv2.convertScaleAbs(image, alpha=1.5, beta=0)
    contrast_bbox = bbox
    contrast_keypoints = keypoints
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # Contrast -50
    contrast_image = cv2.convertScaleAbs(image, alpha=0.5, beta=0)
    contrast_bbox = bbox
    contrast_keypoints = keypoints
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # Horizontal flip
    flipped_image = cv2.flip(image, 1)
    flipped_bbox = flip_bbox(bbox, image.shape[1])
    flipped_keypoints = flip_keypoints(keypoints, image.shape[1])
    augmented_data.append((flipped_image, flipped_bbox, flipped_keypoints))

    # Horizontal flip 30°
    rotated_image = cv2.warpAffine(flipped_image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), 30, 1), (image.shape[1], image.shape[0]))
    rotated_bbox = rotate_bbox(flipped_bbox, 30, image.shape[1], image.shape[0])
    rotated_keypoints = rotate_keypoints(flipped_keypoints, 30, image.shape[1], image.shape[0])
    augmented_data.append((rotated_image, rotated_bbox, rotated_keypoints))

    # Horizontal flip -30°
    rotated_image = cv2.warpAffine(flipped_image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), -30, 1), (image.shape[1], image.shape[0]))
    rotated_bbox = rotate_bbox(flipped_bbox, -30, image.shape[1], image.shape[0])
    rotated_keypoints = rotate_keypoints(flipped_keypoints, -30, image.shape[1], image.shape[0])
    augmented_data.append((rotated_image, rotated_bbox, rotated_keypoints))

    # Contrast Horizontal flip +50
    contrast_image = cv2.convertScaleAbs(flipped_image, alpha=1.5, beta=0)
    contrast_bbox = flip_bbox(flipped_bbox, image.shape[1])
    contrast_keypoints = flip_keypoints(flipped_keypoints, image.shape[1])
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # Contrast Horizontal flip -50
    contrast_image = cv2.convertScaleAbs(flipped_image, alpha=0.5, beta=0)
    contrast_bbox = flip_bbox(flipped_bbox, image.shape[1])
    contrast_keypoints = flip_keypoints(flipped_keypoints, image.shape[1])
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # Color change to red
    red_image = image.copy()
    red_image[:, :, 1] = 0  # Set green channel to 0
    red_image[:, :, 2] = 0  # Set blue channel to 0
    red_bbox = bbox
    red_keypoints = keypoints
    augmented_data.append((red_image, red_bbox, red_keypoints))

    # Color change to green
    green_image = image.copy()
    green_image[:, :, 0] = 0  # Set blue channel to 0
    green_image[:, :, 2] = 0  # Set red channel to 0
    green_bbox = bbox
    green_keypoints = keypoints
    augmented_data.append((green_image, green_bbox, green_keypoints))

    # Color change to purple
    purple_image = image.copy()
    purple_image[:, :, 0] = 128  # Set blue channel to half intensity
    purple_image[:, :, 1] = 0    # Set green channel to 0
    purple_bbox = bbox
    purple_keypoints = keypoints
    augmented_data.append((purple_image, purple_bbox, purple_keypoints))

    # Horizontal zoom
    zoom_factor = 0.8
    zoom_image = cv2.resize(image, None, fx=zoom_factor, fy=1)
    zoomed_bbox = zoom_bbox(bbox, zoom_factor, image.shape[1])
    zoomed_keypoints = zoom_keypoints(keypoints, zoom_factor, image.shape[1])
    augmented_data.append((zoom_image, zoomed_bbox, zoomed_keypoints))

    # Horizontal zoom with flip
    zoom_flip_image = cv2.flip(zoom_image, 1)
    zoom_flip_bbox = flip_bbox(zoomed_bbox, zoom_image.shape[1])
    zoom_flip_keypoints = flip_keypoints(zoomed_keypoints, zoom_image.shape[1])
    augmented_data.append((zoom_flip_image, zoom_flip_bbox, zoom_flip_keypoints))

    # Horizontal zoom with rotate 30°
    rotated_zoom_image = cv2.warpAffine(zoom_image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), 30, 1), (image.shape[1], image.shape[0]))
    rotated_zoom_bbox = rotate_bbox(zoomed_bbox, 30, image.shape[1], image.shape[0])
    rotated_zoom_keypoints = rotate_keypoints(zoomed_keypoints, 30, image.shape[1], image.shape[0])
    augmented_data.append((rotated_zoom_image, rotated_zoom_bbox, rotated_zoom_keypoints))

    # Horizontal zoom with rotate -30°
    rotated_zoom_image = cv2.warpAffine(zoom_image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), -30, 1), (image.shape[1], image.shape[0]))
    rotated_zoom_bbox = rotate_bbox(zoomed_bbox, -30, image.shape[1], image.shape[0])
    rotated_zoom_keypoints = rotate_keypoints(zoomed_keypoints, -30, image.shape[1], image.shape[0])
    augmented_data.append((rotated_zoom_image, rotated_zoom_bbox, rotated_zoom_keypoints))
    
    # Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = cv2.magnitude(sobel_x, sobel_y)
    salient_edge_map = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    salient_edge_bbox = bbox
    salient_edge_keypoints = [kp for kp in keypoints]
    augmented_data.append((salient_edge_map, salient_edge_bbox, salient_edge_keypoints))


    return augmented_data

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_data_list = augment_data(image, bbox, keypoints)

        # Save augmented images and labels
        for i, (augmented_image, augmented_bbox, augmented_keypoints) in enumerate(augmented_data_list):
            augmented_image_path = os.path.join(output_dir, f'aug_{i}_{filename}')
            cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(output_dir, f'aug_{i}_{filename[:-4]}.txt')
            with open(augmented_label_path, 'w') as augmented_label_file:
                augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
                augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")


Data augmentation complete.


In [2]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'data_p\img'
output_dir = 'dataug'
label_dir = 'data_p\lab'

# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    augmented_data = []

    # Original data
    #augmented_data.append((image, bbox, keypoints))

    # 1- Contrast +50
    contrast_image = cv2.convertScaleAbs(image, alpha=1.5, beta=0)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 2- Contrast -50
    contrast_image = cv2.convertScaleAbs(image, alpha=0.5, beta=0)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 3- Contrast +75
    contrast_image = cv2.convertScaleAbs(image, alpha=1.50, beta=-200)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 4- Contrast -75
    contrast_image = cv2.convertScaleAbs(image, alpha=0.75, beta=-190)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))
    
    # 5- Color Rojo
    red_image = image.copy()
    red_image[:, :, 1] = 0  # Set green channel to 0
    red_image[:, :, 2] = 0  # Set blue channel to 0
    red_bbox = bbox
    red_keypoints = [kp for kp in keypoints]
    augmented_data.append((red_image, red_bbox, red_keypoints))

    # 6- Color Verde
    green_image = image.copy()
    green_image[:, :, 0] = 0  # Set blue channel to 0
    green_image[:, :, 2] = 0  # Set red channel to 0
    green_bbox = bbox
    green_keypoints = [kp for kp in keypoints]
    augmented_data.append((green_image, green_bbox, green_keypoints))

    # 7- Color Purpura
    purple_image = image.copy()
    purple_image[:, :, 0] = 128  # Set blue channel to half intensity
    purple_image[:, :, 1] = 0    # Set green channel to 0
    purple_bbox = bbox
    purple_keypoints = [kp for kp in keypoints]
    augmented_data.append((purple_image, purple_bbox, purple_keypoints))

    # 8- Color Amarillo
    purple_image = image.copy()
    purple_image[:, :, 2] = 50 # Set blue channel to half intensity
    purple_image[:, 2, 1] = 128    # Set green channel to 0
    purple_bbox = bbox
    purple_keypoints = [kp for kp in keypoints]
    augmented_data.append((purple_image, purple_bbox, purple_keypoints))

    # 9- Color Azul
    purple_image = image.copy()
    purple_image[2, :, 0] = 128 # Set blue channel to half intensity
    purple_image[:, :, 0] = 0    # Set green channel to 0
    purple_bbox = bbox
    purple_keypoints = [kp for kp in keypoints]
    augmented_data.append((purple_image, purple_bbox, purple_keypoints))
    
    # 10- Contrast y Brillo +50
    contrast_image = cv2.convertScaleAbs(image, alpha=2.0, beta=-150)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))
    
    # 11- Des-Texturizado
    kernel = np.array([[0, 1, 0],
                       [1, -3, 1],
                       [0, 1, 0]], dtype=np.float32)

    edge_enhanced_image = cv2.filter2D(image, cv2.CV_8U, kernel)
    edge_enhanced_bbox = bbox
    edge_enhanced_keypoints = [kp for kp in keypoints]
    augmented_data.append((edge_enhanced_image, edge_enhanced_bbox, edge_enhanced_keypoints))
    
    # 12- Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = cv2.magnitude(sobel_x, sobel_y)
    salient_edge_map = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    salient_edge_bbox = bbox
    salient_edge_keypoints = [kp for kp in keypoints]
    augmented_data.append((salient_edge_map, salient_edge_bbox, salient_edge_keypoints))
    
    # 13- Contrast y Brillo +50
    contrast_image = cv2.convertScaleAbs(image, alpha=1.5, beta=50)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 14- Contrast y Brillo -50
    contrast_image = cv2.convertScaleAbs(image, alpha=0.5, beta=-50)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 15- Decolorar la imagen
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    decolor_bbox = bbox
    decolor_keypoints = [kp for kp in keypoints]
    augmented_data.append((gray_image, decolor_bbox, decolor_keypoints))

    return augmented_data

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_data_list = augment_data(image, bbox, keypoints)

        # Save augmented images and labels
        for i, (augmented_image, augmented_bbox, augmented_keypoints) in enumerate(augmented_data_list):
            augmented_image_path = os.path.join(output_dir, f'aug_{i}_{filename}')
            cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(output_dir, f'aug_{i}_{filename[:-4]}.txt')
            with open(augmented_label_path, 'w') as augmented_label_file:
                augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
                augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")



FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'data_p\\img'

In [6]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'golpes_3_340\images'
output_dir = 'dataug'
label_dir = 'golpes_3_340\labels'


# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Helper functions for bounding box and keypoints transformations
def flip_bbox(bbox, image_width):
    return [image_width - bbox[2], bbox[1], image_width - bbox[0], bbox[3]]

def rotate_bbox(bbox, angle, image_width, image_height):
    center_x = (bbox[0] + bbox[2]) / 2
    center_y = (bbox[1] + bbox[3]) / 2

    new_center_x = center_x * np.cos(np.radians(angle)) - center_y * np.sin(np.radians(angle))
    new_center_y = center_x * np.sin(np.radians(angle)) + center_y * np.cos(np.radians(angle))

    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]

    new_width = np.abs(width * np.cos(np.radians(angle))) + np.abs(height * np.sin(np.radians(angle)))
    new_height = np.abs(height * np.cos(np.radians(angle))) + np.abs(width * np.sin(np.radians(angle)))

    x_min = max(0, new_center_x - new_width / 2)
    y_min = max(0, new_center_y - new_height / 2)
    x_max = min(image_width, new_center_x + new_width / 2)
    y_max = min(image_height, new_center_y + new_height / 2)

    return [x_min, y_min, x_max, y_max]

def rotate_keypoints(keypoints, angle, image_width, image_height):
    rotated_keypoints = []
    center_x = image_width / 2
    center_y = image_height / 2

    for i in range(0, len(keypoints), 2):
        x = keypoints[i]
        y = keypoints[i + 1]

        # Translate to the origin
        x -= center_x
        y -= center_y

        # Rotate
        new_x = x * np.cos(np.radians(angle)) - y * np.sin(np.radians(angle))
        new_y = x * np.sin(np.radians(angle)) + y * np.cos(np.radians(angle))

        # Translate back to the original center
        new_x += center_x
        new_y += center_y

        rotated_keypoints.extend([new_x, new_y])

    return rotated_keypoints


def flip_keypoints(keypoints, image_width):
    flipped_keypoints = [image_width - kp if i % 2 == 0 else kp for i, kp in enumerate(keypoints)]
    return flipped_keypoints

def zoom_bbox(bbox, zoom_factor, image_width):
    center_x = (bbox[0] + bbox[2]) / 2

    new_width = (bbox[2] - bbox[0]) * zoom_factor

    x_min = max(0, center_x - new_width / 2)
    x_max = min(image_width, center_x + new_width / 2)

    return [x_min, bbox[1], x_max, bbox[3]]  # Fix bbox[0] to bbox[1]


def zoom_keypoints(keypoints, zoom_factor, image_width):
    center_x = (keypoints[0] + keypoints[2]) / 2

    new_x_0 = center_x - (center_x - keypoints[0]) * zoom_factor
    new_x_2 = center_x + (keypoints[2] - center_x) * zoom_factor

    return [new_x_0, keypoints[1], new_x_2, keypoints[3]]  # Fix keypoints[0] to keypoints[1]


# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    augmented_data = []

    # Original data
    #augmented_data.append((image, bbox, keypoints))

    # Rotate 30°
    rotated_image = cv2.warpAffine(image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), 15, 1), (image.shape[1], image.shape[0]))
    rotated_bbox = rotate_bbox(bbox, -15, image.shape[1], image.shape[0])
    rotated_keypoints = rotate_keypoints(keypoints, -15, image.shape[1], image.shape[0])
    augmented_data.append((rotated_image, rotated_bbox, rotated_keypoints))

    # Rotate -30°
    #rotated_image = cv2.warpAffine(image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), -30, 1), (image.shape[1], image.shape[0]))
    #rotated_bbox = rotate_bbox(bbox, -30, image.shape[1], image.shape[0])
    #rotated_keypoints = rotate_keypoints(keypoints, -30, image.shape[1], image.shape[0])
    #augmented_data.append((rotated_image, rotated_bbox, rotated_keypoints))




    return augmented_data

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_data_list = augment_data(image, bbox, keypoints)

        # Save augmented images and labels
        for i, (augmented_image, augmented_bbox, augmented_keypoints) in enumerate(augmented_data_list):
            augmented_image_path = os.path.join(output_dir, f'aug_{i}_{filename}')
            cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(output_dir, f'aug_{i}_{filename[:-4]}.txt')
            with open(augmented_label_path, 'w') as augmented_label_file:
                augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
                augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")


KeyboardInterrupt: 

In [46]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'golpes_3_340\images'
output_dir = 'dataug'
label_dir = 'golpes_3_340\labels'

# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Helper functions for bounding box and keypoints transformations


def rotate_bbox(bbox, angle, image_width, image_height):
    center_x = (bbox[0] + bbox[2]) / 2
    center_y = (bbox[1] + bbox[3]) / 2

    # Calcular las nuevas coordenadas del centro después de la rotación
    new_center_x = center_x * np.cos(np.radians(angle)) - center_y * np.sin(np.radians(angle))
    new_center_y = center_x * np.sin(np.radians(angle)) + center_y * np.cos(np.radians(angle))

    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]

    # Calcular las nuevas dimensiones de la bounding box después de la rotación
    new_width = np.abs(width * np.cos(np.radians(angle))) + np.abs(height * np.sin(np.radians(angle)))
    new_height = np.abs(height * np.cos(np.radians(angle))) + np.abs(width * np.sin(np.radians(angle)))

    # Calcular las nuevas coordenadas de la bounding box después de la rotación
    x_min = max(0, new_center_x - new_width / 2)
    y_min = max(0, new_center_y - new_height / 2)
    x_max = min(image_width, new_center_x + new_width / 2)
    y_max = min(image_height, new_center_y + new_height / 2)

    return [x_min, y_min, x_max, y_max]

def rotate_keypoints(keypoints, angle, image_width, image_height):
    rotated_keypoints = []

    # Iterar sobre las coordenadas x, y de los keypoints
    for i in range(0, len(keypoints), 2):
        x = keypoints[i]
        y = keypoints[i + 1]

        # Calcular las nuevas coordenadas después de la rotación
        new_x = x * np.cos(np.radians(angle)) - y * np.sin(np.radians(angle))
        new_y = x * np.sin(np.radians(angle)) + y * np.cos(np.radians(angle))

        rotated_keypoints.extend([new_x, new_y])

    return rotated_keypoints

# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    augmented_data = []

    # Original data
    # augmented_data.append((image, bbox, keypoints))

    # Rotate -15°
    rotated_image = cv2.warpAffine(image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), -15, 1), (image.shape[1], image.shape[0]))
    rotated_bbox = rotate_bbox(bbox, 15, image.shape[1], image.shape[0])
    rotated_keypoints = rotate_keypoints(keypoints, 15, image.shape[1], image.shape[0])
    augmented_data.append((rotated_image, rotated_bbox, rotated_keypoints))

    return augmented_data

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_data_list = augment_data(image, bbox, keypoints)

        # Save augmented images and labels
        for i, (augmented_image, augmented_bbox, augmented_keypoints) in enumerate(augmented_data_list):
            augmented_image_path = os.path.join(output_dir, f'aug_{i}_{filename}')
            cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(output_dir, f'aug_{i}_{filename[:-4]}.txt')
            with open(augmented_label_path, 'w') as augmented_label_file:
                augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
                augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")



KeyboardInterrupt: 

In [21]:
import cv2
import numpy as np
import os

def draw_bbox_and_keypoints(image,  keypoints, color=(255, 0, 0)):

    # Draw keypoints
    for i in range(0, len(keypoints), 2):
        kp_x = int(keypoints[i] * image.shape[1])
        kp_y = int(keypoints[i + 1] * image.shape[0])
        cv2.circle(image, (kp_x, kp_y), 3, color, -1)

    return image

# Define your input directory
input_dir = 'data\images\train'

label_dir = 'dataug'

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Draw bounding box and keypoints on the image
        image_with_annotations = draw_bbox_and_keypoints(image.copy(), keypoints)

        # Display the result
        cv2.imshow('Annotations Verification', cv2.cvtColor(image_with_annotations, cv2.COLOR_RGB2BGR))
        cv2.waitKey(0)
        cv2.destroyAllWindows()


KeyboardInterrupt: 

In [8]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'nose\img'
output_dir = 'dataug'
label_dir = 'nose\lab'

# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    augmented_data = []

    # Original data
    #augmented_data.append((image, bbox, keypoints))

    # Horizontal flip
    flipped_image = cv2.flip(image, 1)
    flipped_bbox = [1 - bbox[2], bbox[1], 1 - bbox[0], bbox[3]]  # Adjust bounding box x-coordinates
    flipped_keypoints = [1 - kp if i % 2 == 0 else kp for i, kp in enumerate(keypoints)]
    augmented_data.append((flipped_image, flipped_bbox, flipped_keypoints))



    


    return augmented_data

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_data_list = augment_data(image, bbox, keypoints)

        # Save augmented images and labels
        for i, (augmented_image, augmented_bbox, augmented_keypoints) in enumerate(augmented_data_list):
            augmented_image_path = os.path.join(output_dir, f'aug_{i}_{filename}')
            cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(output_dir, f'aug_{i}_{filename[:-4]}.txt')
            with open(augmented_label_path, 'w') as augmented_label_file:
                augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
                augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")



Data augmentation complete.


In [10]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'golpes_3_340\images'
output_dir = 'dataug'
label_dir = 'golpes_3_340\labels'

# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    augmented_data = []

    # Original data
    #augmented_data.append((image, bbox, keypoints))
    # Contrast +50
    contrast_image = cv2.convertScaleAbs(image, alpha=1.5, beta=0)
    contrast_bbox = bbox
    contrast_keypoints = keypoints
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # Contrast -50
    contrast_image = cv2.convertScaleAbs(image, alpha=0.5, beta=0)
    contrast_bbox = bbox
    contrast_keypoints = keypoints
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    return augmented_data

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_data_list = augment_data(image, bbox, keypoints)

        # Save augmented images and labels
        for i, (augmented_image, augmented_bbox, augmented_keypoints) in enumerate(augmented_data_list):
            augmented_image_path = os.path.join(output_dir, f'aug_{i}_{filename}')
            cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(output_dir, f'aug_{i}_{filename[:-4]}.txt')
            with open(augmented_label_path, 'w') as augmented_label_file:
                augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
                augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")



KeyboardInterrupt: 

In [55]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'golpes_3_340\images'
output_dir = 'dataug'
label_dir = 'golpes_3_340\labels'

# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    augmented_data = []

    # Original data
    #augmented_data.append((image, bbox, keypoints))

    # 3- Contrast +75
    contrast_image = cv2.convertScaleAbs(image, alpha=2.0, beta=-150)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    return augmented_data

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_data_list = augment_data(image, bbox, keypoints)

        # Save augmented images and labels
        for i, (augmented_image, augmented_bbox, augmented_keypoints) in enumerate(augmented_data_list):
            augmented_image_path = os.path.join(output_dir, f'aug_{i}_{filename}')
            cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(output_dir, f'aug_{i}_{filename[:-4]}.txt')
            with open(augmented_label_path, 'w') as augmented_label_file:
                augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
                augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")


KeyboardInterrupt: 